In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
# Days and screens
DAYS = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
SCREENS = ["Screen_1", "Screen_2", "Screen_3"]

# Operating hours per screen per day (e.g., 10 hours = 600 minutes)
OPERATING_MIN_PER_DAY = {
    day: 10 * 60 for day in DAYS  # same for all days for now
}

# Theater capacities per screen
SCREEN_CAPACITY = {
    "Screen_1": 120,
    "Screen_2": 80,
    "Screen_3": 60,
}

# Buffer time between shows (e.g., cleaning, trailers, turnover) in minutes
BUFFER_MIN = 15

# Max number of showings per movie per day (frequency constraint)
MAX_SHOWINGS_PER_MOVIE_PER_DAY = 5

# Synthetic movie Data 
movies_df = pd.DataFrame({
    "movie_id": ["M1", "M2", "M3", "M4"],
    "title": ["Blockbuster Action", "Indie Drama", "Family Animation", "Horror Thriller"],
    "runtime_min": [130, 110, 95, 100],
    "ticket_price": [15.0, 14.0, 13.0, 13.0]
})

# Optimization Formulation

1. Parameters

- $A_i,d$: expected attendance for movie i per day d

- $P_d$: average ticket price for movie on day d
- $D_i$: duration of movie i in minutes
- $B$: buffer time between shows
- $C_j$: capacity of screen j
- $H_j,d$: hours screen j operates on day d 
- $K$: maximum times a movie can be shown on a given day


2. Decision Variables:

- $s_i,d$: { 0, 1 }: whether movie i is scheduled on day d
- $x_i, d, j$: number of times we show movie i on screen j on a given day d 
- $r_i, d$: realized tickets served for movie i on day d, where $r_i,d=\min(j  x_i, d,j C_j,  A_i,d) $

3. Objective function: Maximize daily revenue:

$$max (i, d,j\times P_d r_i,d   ) $$


4. Constraints
- Total screen time: $\sum_{j} x_{i,d,j} (D_i+B) \leq H_{d,j} \forall d,j$
- Tickets served won’t exceed demand:  $r_i,d\leq \sum_{j} x_i, d,j C_j$
- Can’t sell more tickets than available: $r_i,d \leq A_i,d$  
- If not scheduled, no showings:  $\sum_{j} x_i, d,j \leq M*s_i,d$
- Each movie shown at most K times: $\sum_{j} x_i, d,j \leq K  ∀ i,d$

In [ ]:
from audioop import maxpp


def createModel(runtimes, demand, ticket_price, max_showings_per_day, operating_min_per_day, screen_capacity, movie_ids, days, screens, buffer_min, verbose=False):

    m = gp.Model("CinemaShowtimeScheduling")
    if not verbose:
        m.Params.OutputFlag = 0  # silence output by default
    
    # Decision variables
    # x[i,d,j] >= 0 integer
    x = m.addVars(
        movie_ids, days, screens,
        vtype=GRB.INTEGER,
        name="x"
    )
    
    # r[i,d] >= 0 continuous
    r = m.addVars(
        movie_ids, days,
        vtype=GRB.CONTINUOUS,
        lb=0.0,
        name="r"
    )
    
    # s[i,d] binary
    s = m.addVars(
        movie_ids, days,
        vtype=GRB.BINARY,
        name="s"
    )
    
    # Objective: maximize total revenue
    m.setObjective(
        gp.quicksum(ticket_price[i] * r[i, d] for i in movie_ids for d in days),
        GRB.MAXIMIZE
    )
    
    # 1) Screen time constraint per screen & day
    #   sum_i x[i,d,j] * (runtime_i + buffer) <= operating_min_per_day[d]
    for d in days:
        for j in screens:
            m.addConstr(
                sum(x[i, d, j] * (runtimes[i] + buffer_min) for i in movie_ids)
                <= operating_min_per_day[d],
                name=f"Time_{d}_{j}"
            )
    
    # 2) Capacity: r[i,d] <= sum_j x[i,d,j] * capacity_j
    for i in movie_ids:
        for d in days:
            m.addConstr(
                r[i, d] <= gp.quicksum(x[i, d, j] * screen_capacity[j] for j in screens),
                name=f"Capacity_{i}_{d}"
            )
    
    # 3) Demand: r[i,d] <= A[i,d]
    for i in movie_ids:
        for d in days:
            demand_id = demand.get((i, d), 0)
            m.addConstr(
                r[i, d] <= demand_id,
                name=f"Demand_{i}_{d}"
            )
    
    # 4) Frequency + schedule linking
    for i in movie_ids:
        for d in days:
            # Frequency limit: total showings per day
            # Link to s: if s[i,d] = 0 => no showings
            m.addConstr(
                sum(x[i, d, j] for j in screens)
                <= max_showings_per_day * s[i, d],
                name=f"FreqLimit_{i}_{d}"
            )
         
    return m
    

In [ ]:
model = None

In [ ]:
model.optimize()

In [ ]:
def optimize_showtimes(model, verbose = False):

    model.optimize()


    status_code = m.Status
    if status_code == GRB.OPTIMAL:
        status = "Optimal"
    elif status_code == GRB.INFEASIBLE:
        status = "Infeasible"
    elif status_code == GRB.UNBOUNDED:
        status = "Unbounded"
    else:
        status = f"Status_{status_code}"
    
    total_revenue = m.objVal if status_code == GRB.OPTIMAL else None
    
    if verbose:
        print("Status:", status)
        print("Total revenue:", total_revenue)
    
    schedule_rows = []
    if status_code == GRB.OPTIMAL:
        for i in movie_ids:
            for d in days:
                for j in screens:
                    val = x[i, d, j].X
                    if val > 1e-6:
                        schedule_rows.append({
                            "movie_id": i,
                            "day": d,
                            "screen": j,
                            "showings": int(round(val))
                        })
    
    schedule_df = pd.DataFrame(schedule_rows)
    
    realized_rows = []
    if status_code == GRB.OPTIMAL:
        for i in movie_ids:
            for d in days:
                realized_rows.append({
                    "movie_id": i,
                    "day": d,
                    "realized_tickets": r[i, d].X,
                    "scheduled_flag": int(round(s[i, d].X))
                })
    realized_df = pd.DataFrame(realized_rows)
    
    return {
        "status": status,
        "total_revenue": total_revenue,
        "schedule_df": schedule_df,
        "realized_df": realized_df
    }
    